## SProcLib Quick Start Example

In [ ]:
!pip install sproclib

In [ ]:
# Import essential sproclib components
from sproclib.unit.plant import ChemicalPlant
from sproclib.unit.pump import CentrifugalPump
from sproclib.unit.reactor import CSTR

In [ ]:
# Create plant
plant = ChemicalPlant(name="Quick Start Plant")

# Add units
plant.add(CentrifugalPump(H0=50.0, eta=0.75), name="pump")
plant.add(CSTR(V=150.0, k0=7.2e10), name="reactor")

# Connect units
plant.connect("pump", "reactor", "feed_stream")

# Configure optimization
plant.compile(optimizer="economic", loss="total_cost")

## Optimize and Display Results

Run optimization and show the results:

In [ ]:
# Optimize the plant
results = plant.optimize(target_production=1000.0)

# Display results
print("Optimization Results:")
print(f"Success: {results['success']}")
if results['success']:
    print(f"Optimal Cost: ${results['optimal_cost']:.2f}")
    print(f"Message: {results['message']}")

## Plant Summary and Evaluation

Display plant summary and performance metrics:

In [ ]:
# Show plant summary
plant.summary()

# Evaluate performance
evaluation = plant.evaluate({})
print("\nPlant Performance:")
if 'plant' in evaluation:
    for metric, value in evaluation['plant'].items():
        print(f"  {metric}: {value:.2f}" if isinstance(value, float) else f"  {metric}: {value}")